In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cProfile import label #?not sure
import re
from fuzzywuzzy import fuzz

import difflib 
from fuzzywuzzy import process


### Feb 17 todo list:

- import basics from extract flora code
- organize fuzzy matching authors 
    - function for splitting authors into sets
    - functions for exact matching species name and fuzzy matching authors
    - functions for getting the match types 

### importing books and WCVP

In [2]:
# to fix unnecessary space at the end of authors (will fix index files later)
def fix_author(row):
    if not isinstance(row, str) or row[-1] != ' ':
        return row
    return row[:-1]

In [7]:
vol1_path = '../input/input copy/index_without_itallics_csvs/vol1/vol1_index_pruned.csv'
vol2_path = '../input/input copy/index_without_itallics_csvs/vol2/vol2_index_pruned.csv'
vol3_path = '../input/input copy/index_without_itallics_csvs/vol3/vol3_index_pruned.csv'

vol1_df = pd.read_csv(vol1_path)
vol2_df = pd.read_csv(vol2_path)
vol3_df = pd.read_csv(vol3_path)

#fix spacing issue
vol1_df['author'] = vol1_df['author'].apply(fix_author)
vol2_df['author'] = vol2_df['author'].apply(fix_author)
vol3_df['author'] = vol3_df['author'].apply(fix_author)

#changing name of columns 
vol1_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol2_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol3_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)

#wcvp 
text_wcvp_path = '../input/input copy/wcvp/wcvp_v9_jun_2022.txt'
wcvp_df = pd.read_csv(text_wcvp_path, sep = '|')

### process wcvp (explode authors)

In [207]:
def get_set(x):
    if not isinstance(x, list): #is this the best way
        x = []
    return set(x)

#to use after exploding the authors
connectors = {'ex', '&', 'et'}
def remove_prantheses(x):
    if isinstance(x, str):
        return re.sub(r'([()])', '', x)
    return x

def process_wcvp(wcvp_df):
    wcvp_df['infraspecies'].replace(np.nan, 'np.NaN', inplace=True)
    wcvp_df['species'].replace(np.nan, 'np.NaN', inplace=True)
    wcvp_df['authors_set'] = wcvp_df['authors'].str.split(' ').apply(lambda x : get_set(x) - connectors)
    wcvp_df_explode = wcvp_df.explode('authors_set')
    wcvp_df_explode['authors_set_'] = wcvp_df_explode['authors_set'].apply(remove_prantheses) #to remove prantheses
    return wcvp_df_explode

wcvp_df_explode = process_wcvp(wcvp_df)

### process vols (explore authors)

just doing the set process during matching

In [23]:
"""def process_vols(df):
    delimeter = ' ' #TODO: FIX this absolutely HORRIBLE delimeter 
    df['authors_set'] = df['mouterde_author'].str.split(delimeter).apply(lambda x : get_set(x) - connectors)
    df_explode = df.explode('authors_set')
    df_explode['authors_set'] = df_explode['authors_set'].apply(remove_prantheses) #to remove prantheses
    return df_explode"""

In [26]:
"""vol1_explode = process_vols(vol1_df)
vol2_explode = process_vols(vol2_df)
vol3_explode = process_vols(vol3_df)"""

### Fixing Mout. authors

#### TODO: PUT THIS IN RIGHT ORDER reimports 

In [104]:
vol1_path = '../input/input copy/index_without_itallics_csvs/vol1/vol1_index_pruned.csv'
vol2_path = '../input/input copy/index_without_itallics_csvs/vol2/vol2_index_pruned.csv'
vol3_path = '../input/input copy/index_without_itallics_csvs/vol3/vol3_index_pruned.csv'

vol1_df = pd.read_csv(vol1_path)
vol2_df = pd.read_csv(vol2_path)
vol3_df = pd.read_csv(vol3_path)

#fix spacing issue
vol1_df['author'] = vol1_df['author'].apply(fix_author)
vol2_df['author'] = vol2_df['author'].apply(fix_author)
vol3_df['author'] = vol3_df['author'].apply(fix_author)

#changing name of columns 
vol1_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol2_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol3_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)

#wcvp 
text_wcvp_path = '../input/input copy/wcvp/wcvp_v9_jun_2022.txt'
wcvp_df = pd.read_csv(text_wcvp_path, sep = '|')

In [105]:
vol1_df['mouterde_author']

,page_num,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon rank
0,616,Acanthophyllum,NaN,NaN,NaN,genus
1,616,Acanthophyllum,kurdicum,NaN,Boiss. et Hausskn.,species
2,616,Achyranthes,NaN,NaN,NaN,genus
3,616,Achyranthes,aspera,NaN,L.,species
4,616,Adiantum,NaN,NaN,NaN,genus
...,...,...,...,...,...,...
1382,638,Vulpia,uniglumis,NaN,(Sol.) Dumort.,species
1383,638,Zannichellia,NaN,NaN,NaN,genus
1384,638,Zannichellia,palustris,NaN,L.,species
1385,638,Zostera,NaN,NaN,NaN,genus


#### regex part 

regex for close pranthesis: 
- .+\)
    - split the string into in and out of paranthesis
regex for anything that comes after ex (with two space around it):
- \sex[\s|.].*
    - remove everything after ex (in each set created by end paranthesis)
- \set[\s|.]|\sin[\s|.]|\snon[\s|.]
    - split each set (in each set created by end paranthesis) by et, in, non after that

In [213]:
#using regex above
def split_author(row):
    if (not isinstance(row, str)) or row == "np.NaN":
        return row
    paran_list = []
    reg_list = []

    in_pran = ""
    out_pran = ""
    #the paranthesis part
    search_full_paran = re.search(r"\(.*\)", row)
    search_close_paran = re.search(r".*\)", row)
    if search_full_paran:
        in_pran = search_full_paran.group(0)[1:-1] #assuming we have at most 1 part of () prob use sub later
    elif search_close_paran:
        in_pran = search_full_paran.group(0)[:-1]
    
    split_paran = re.split(r".*\)", row, maxsplit=1) #not sure if this removes anything after last or not 
    out_pran = split_paran[int(len(split_paran) == 2)]
    
    #remove anything after ex. in out_pran and in_pran
    in_pran = re.sub(r"\sex[\s|.].*","",in_pran)
    out_pran = re.sub(r"\sex[\s|.].*","",out_pran)

    #split by et, in non with possible . after them
    paran_list = re.split(r"\set[\s|.]|\sin[\s|.]|\snon[\s|.]|\s&[\s|.]", in_pran) #WILL NOT REMOVE IF et AT END
    reg_list = re.split(r"\set[\s|.]|\sin[\s|.]|\snon[\s|.]|\s&[\s|.]", out_pran) 
    
    return [aut for aut in paran_list + reg_list if aut != ""] #bandaid lol but it's still very fast so doesn't matter really

In [214]:
vol3_df['mouterde_author_list'] = vol3_df['mouterde_author'].apply(split_author)

In [215]:
vol3_df.tail(50)

,page_num,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon rank,mouterde_author_list
1572,581,Veronica,panormitana,Fischer,NaN,A.,NaN
1573,581,Veronica,panormitana,panormitana,NaN,subsp.,NaN
1574,581,Veronica,persica,NaN,Poir.,species,[Poir.]
1575,581,Veronica,polifolia,NaN,Benth.,species,[Benth.]
1576,581,Veronica,poliţa,NaN,Fries,species,[Fries]
1577,581,Veronica,pusilla,NaN,Kotschy et Boiss.,species,"[Kotschy, Boiss.]"
1578,581,Veronica,reuterana,NaN,Boiss.,species,[Boiss.]
1579,581,Veronica,samuelssonii,NaN,Rech.,species,[Rech.]
1580,581,Veronica,scardica,NaN,Griseb. lysimachioides Boiss.,species,[Griseb. lysimachioides Boiss.]
1581,581,Veronica,syriaca,NaN,Roem. et Sch.,species,"[Roem., Sch.]"


### Get Closest Match

## UNTITLED 

#### feb 19

tldr : need to update wcvp author set now to work similar to the mout authors in regex
- added & to account for variations (et vs &). ex works as expected. Not sure about other symbols tho ... 

In [219]:
wcvp_df['author_list'] = wcvp_df['authors'].apply(split_author)

In [218]:
wcvp_df.head(50)


,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,authors_set,author_list
0,338-1,Acanthaceae,Acanthodium,np.NaN,np.NaN,Acanthodium,Delile,GENUS,Synonym,427-1,Blepharis,Juss.,NaN,NaN,NaN,In review,"Descr. Egypte, Hist. Nat. 2(Mém.): 241 (1813)",NaN,{Delile},[Delile]
1,44787-1,Acanthaceae,Acanthodium,angustum,np.NaN,Acanthodium angustum,Nees,SPECIES,Homotypic_Synonym,46469-1,Blepharis angusta,(Nees) T.Anderson,NaN,NaN,NaN,In review,"A.P.de Candolle, Prodr. 11: 273 (1847)",NaN,{Nees},[Nees]
2,44788-1,Acanthaceae,Acanthodium,capense,np.NaN,Acanthodium capense,(L.f.) Nees,SPECIES,Homotypic_Synonym,46487-1,Blepharis capensis,(L.f.) Pers.,NaN,NaN,NaN,In review,Linnaea 15: 361 (1841),NaN,"{Nees, (L.f.)}","[L.f., Nees]"
3,44789-1,Acanthaceae,Acanthodium,carduifolium,np.NaN,Acanthodium carduifolium,(L.f.) Nees,SPECIES,Homotypic_Synonym,44830-1,Acanthopsis carduifolia,(L.f.) Schinz,NaN,NaN,NaN,In review,"A.P.de Candolle, Prodr. 11: 278 (1847)",44848-1,"{Nees, (L.f.)}","[L.f., Nees]"
4,44790-1,Acanthaceae,Acanthodium,delilii,np.NaN,Acanthodium delilii,H.Buek,SPECIES,Synonym,46503-1,Blepharis edulis,(Forssk.) Pers.,NaN,NaN,NaN,In review,Gen. Sp. Candoll. 3: 1 (1858),NaN,{H.Buek},[H.Buek]
5,44792-1,Acanthaceae,Acanthodium,diversispinum,np.NaN,Acanthodium diversispinum,Nees,SPECIES,Homotypic_Synonym,46501-1,Blepharis diversispina,(Nees) C.B.Clarke,NaN,NaN,NaN,In review,"A.P.de Candolle, Prodr. 11: 275 (1847)",NaN,{Nees},[Nees]
6,44793-1,Acanthaceae,Acanthodium,furcatum,np.NaN,Acanthodium furcatum,(L.f.) Nees,SPECIES,Homotypic_Synonym,46514-1,Blepharis furcata,(L.f.) Pers.,NaN,NaN,NaN,In review,"A.P.de Candolle, Prodr. 11: 276 (1847)",44862-1,"{Nees, (L.f.)}","[L.f., Nees]"
7,44794-1,Acanthaceae,Acanthodium,glabrum,np.NaN,Acanthodium glabrum,Nees,SPECIES,Homotypic_Synonym,77177568-1,Acanthopsis glabra,(Nees) H.M.Steyn,NaN,NaN,NaN,In review,"A.P.de Candolle, Prodr. 11: 278 (1847)",NaN,{Nees},[Nees]
8,44795-1,Acanthaceae,Acanthodium,glaucum,np.NaN,Acanthodium glaucum,Nees,SPECIES,Homotypic_Synonym,44832-1,Acanthopsis glauca,(Nees) Schinz,NaN,NaN,NaN,In review,"A.P.de Candolle, Prodr. 11: 277 (1847)",NaN,{Nees},[Nees]
9,44796-1,Acanthaceae,Acanthodium,grossum,np.NaN,Acanthodium grossum,Wight,SPECIES,Synonym,46587-1,Blepharis scindica,Stocks ex T.Anderson,NaN,NaN,NaN,In review,Icon. Pl. Ind. Orient. 4: t. 1535 (1850),NaN,{Wight},[Wight]


In [209]:
wcvp_df_explode[wcvp_df_explode['genus'] == 'Ziziphora']

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,authors_set,authors_set_
667550,21306-1,Lamiaceae,Ziziphora,np.NaN,np.NaN,Ziziphora,L.,GENUS,Accepted,NaN,NaN,NaN,NaN,NaN,NaN,Reviewed,Sp. Pl.: 21 (1753),NaN,L.,L.
667551,461958-1,Lamiaceae,Ziziphora,abd-el-asisii,np.NaN,Ziziphora abd-el-asisii,Hand.-Mazz.,SPECIES,Synonym,77170162-1,Ziziphora taurica subsp. taurica,NaN,NaN,NaN,NaN,Reviewed,Ann. K. K. Naturhist. Hofmus. 27: 418 (1913),NaN,Hand.-Mazz.,Hand.-Mazz.
667552,77248417-1,Lamiaceae,Ziziphora,acinoides,np.NaN,Ziziphora acinoides,Willk. & Lange,SPECIES,Synonym,60468599-2,Ziziphora aragonensis,Pau,NaN,NaN,NaN,Reviewed,Prodr. Fl. Hispan. 2: 428 (1868),NaN,Lange,Lange
667552,77248417-1,Lamiaceae,Ziziphora,acinoides,np.NaN,Ziziphora acinoides,Willk. & Lange,SPECIES,Synonym,60468599-2,Ziziphora aragonensis,Pau,NaN,NaN,NaN,Reviewed,Prodr. Fl. Hispan. 2: 428 (1868),NaN,Willk.,Willk.
667553,461960-1,Lamiaceae,Ziziphora,acinoides,np.NaN,Ziziphora acinoides,Pall.,SPECIES,Synonym,77171296-1,Ziziphora clinopodioides subsp. clinopodioides,NaN,NaN,NaN,NaN,Reviewed,Reise Russ. Reich. 3: 362 (1776),NaN,Pall.,Pall.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667666,934226-1,Lamiaceae,Ziziphora,vichodceviana,np.NaN,Ziziphora vichodceviana,Tkatsch. ex Tulyag.,SPECIES,Accepted,NaN,NaN,NaN,21306-1,Ziziphora,L.,Reviewed,Opred. Rast. Sred. Azii 9: 362 (1987),NaN,Tkatsch.,Tkatsch.
667666,934226-1,Lamiaceae,Ziziphora,vichodceviana,np.NaN,Ziziphora vichodceviana,Tkatsch. ex Tulyag.,SPECIES,Accepted,NaN,NaN,NaN,21306-1,Ziziphora,L.,Reviewed,Opred. Rast. Sred. Azii 9: 362 (1987),NaN,Tulyag.,Tulyag.
667667,77165291-1,Lamiaceae,Ziziphora,villosa,np.NaN,Ziziphora villosa,(Pers.) Melnikov,SPECIES,Synonym,445709-1,Clinopodium acinos,(L.) Kuntze,NaN,NaN,NaN,Reviewed,Bot. Zhurn. (Moscow & Leningrad) 101: 89 (2016),444245-1,(Pers.),Pers.
667667,77165291-1,Lamiaceae,Ziziphora,villosa,np.NaN,Ziziphora villosa,(Pers.) Melnikov,SPECIES,Synonym,445709-1,Clinopodium acinos,(L.) Kuntze,NaN,NaN,NaN,Reviewed,Bot. Zhurn. (Moscow & Leningrad) 101: 89 (2016),444245-1,Melnikov,Melnikov


### earlier 

In [101]:
def get_closest_match(r):
    m_author = r['mouterde_author'] 
    genus = r['mouterde_genus']
    epithet = r['mouterde_epithet']
    infra = r['mouterde_infra']
    wcvp_df_gs_match = wcvp_df[(wcvp_df['genus'] == genus) & (wcvp_df['species'] == epithet) & (wcvp_df['infraspecies'] == infra)]
    
    possible_author = []

    #this doesn't work as is but here's the idea 
    
    if isinstance(m_author, str):
        m_author_set = list(get_set(m_author.split(' ')) - connectors)
        for author in m_author_set:
            #print(author, process.extract(author, wcvp_df_gs_match['authors_set']))
            try:
                author_match, author_score, _ = process.extract(author, wcvp_df_gs_match['authors_set'])[0]
                print(author, author_match, author_score)
                if author_score >= 85:
                    possible_author.append(author_match)
            except:
                possible_author.append(np.nan)
        #author_match = difflib.get_close_matches(m_author, wcvp_df_gs_match['authors'])
    else:
        possible_author.append(np.nan)
    return possible_author

In [102]:
vol3_df['closest_wcvp_match'] = vol3_df.apply(get_closest_match, axis = 1)


Less.' {'Less.'} 100
Less. {'Less.'} 100
Boiss. {'Boiss.'} 100
(Willd.) {'(Willd.)', 'Boiss.'} 90
Boiss. {'(Willd.)', 'Boiss.'} 90
Mout. {'Mouterde'} 90
Mobayen {'Mobayen'} 100
Boiss. {'Boiss.'} 100
(Schultes) {'(Willd.', 'Boiss.', 'Schult.)'} 77
Boiss. {'(Willd.', 'Boiss.', 'Schult.)'} 90
L. {'L.'} 100
L. {'L.'} 100
L. {'L.'} 100
Boiss. {'Boiss.'} 100
L. {'L.'} 100
DC. {'DC.'} 100
np.NaN set() 0
Afan. {'Afanasiev'} 90
DC. {'DC.'} 100
L. {'L.'} 100
Schultz {'Sch.Bip.', '(Forssk.)'} 51
Bip. {'Sch.Bip.', '(Forssk.)'} 90
(Forssk.) {'Sch.Bip.', '(Forssk.)'} 90
Boiss. {'Boiss.'} 100
(Labili.). {'DC.'} 0
DC. {'DC.'} 100
DC. {'DC.'} 100
Koch {'K.Koch'} 90
C. {'K.Koch'} 90
Cass. {'Cass.'} 100
Cass. {'Cass.', '(L.)'} 90
(L.) {'Cass.', '(L.)'} 60
L. {'L.'} 100
Davis {'P.H.Davis'} 90
Schreb. {'Schreb.'} 100
Boiss. {'Boiss.'} 100
L. {'L.'} 100
Bilik {'Bilik'} 100
Benth. {'Benth.', 'Ging.'} 90
Gingins {'Benth.', 'Ging.'} 66
Tausch {'Tausch'} 100
Sam. {'Sam.', 'Rech.f.'} 90
Rech. {'Sam.', 'Rech.f.'}

KeyboardInterrupt: 

In [87]:
vol1_df['closest_wcvp_match'] = vol1_df.apply(get_closest_match, axis = 1)
print("vol1 complete")
vol2_df['closest_wcvp_match'] = vol2_df.apply(get_closest_match, axis = 1)
print("vol2 complete")
vol3_df['closest_wcvp_match'] = vol3_df.apply(get_closest_match, axis = 1)
print("vol3 complete")

vol1 complete
vol2 complete
vol3 complete
